Copyright 2022 The Google Research Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and limitations under the License.

In [ ]:
# Imports
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

# Parameters
FILENAMES = [
    '/content/positive_with_drop.tfrecord', '/content/negative.tfrecord'
]
SAVED_MODEL_FOLDER = '/content/saved_model'
POSITIVE_DATA_LABEL_NAME = 'Standup_chair'
CLASS_FEATURE_KEY_ID = 'standup_chair_mapping/label'
CLASS_FEATURE_KEY_NAME = 'standup_chair_mapping/label/name'
DATA_FEATURE_NAME = 'SENSOR/ACCELEROMETER_WINDOW/floats'
DIM_FEATURE_NAME = 'SENSOR/ACCELEROMETER_WINDOW/dimensionality'
WINDOW_SIZE_FEATURE_NAME = 'SENSOR/ACCELEROMETER_WINDOW/window_size'
PERCENT_TRAIN = 0.6
PERCENT_VALIDATION = 0.1
PERCENT_TEST = 0.3
DATASET_BATCH_SIZE = 32

In [ ]:
# Extract dataset from records
dataset = tf.data.TFRecordDataset(FILENAMES)

# Extracted items from dataset, with dict key representing the label ID of the
# data.
labeled_dataset = defaultdict(list)

# Extract sequence exampls
for record in dataset:
  sequence_example = tf.train.SequenceExample()
  sequence_example.ParseFromString(record.numpy())

  label_id = -1
  max_label_id = 0

  # Find label id, accel dimentionality, and accel window size.
  dims = 0
  window_size = -1
  for feature_name, feature in sequence_example.context.feature.items():
    if feature_name == CLASS_FEATURE_KEY_ID:
      label_id = feature.int64_list.value[0]
    if feature_name == DIM_FEATURE_NAME:
      dims = feature.int64_list.value[0]
    if feature_name == WINDOW_SIZE_FEATURE_NAME and window_size < 1:
      window_size = feature.int64_list.value[0]

  if dims == 0:
    print('WARNING: Got sequence example without dimensions for'
          'ACCELEROMETER_WINDOW')
    continue

  if label_id < 0:
    print('WARNING: Label ID less than 0')
    continue

  if label_id > max_label_id:
    max_label_id = label_id

  for (feature_name,
       feature_list) in sequence_example.feature_lists.feature_list.items():
    if feature_name != DATA_FEATURE_NAME:
      continue

    num_features = len(feature_list.feature)
    if window_size > 0 and num_features % window_size != 0:
      print('WARNING: Window size misaligned with number of features')
      continue

    if window_size < 0:
      # Each element in the dataset is a sensor datapoint.
      full_trace = []
      for feature in feature_list.feature:
        if len(feature.float_list.value) != dims:
          print('WARNING: Got data with incorrect dimensions')
          continue
        full_trace.extend(feature.float_list.value)
      labeled_dataset[label_id].append(full_trace)
    else:
      # Each element in the dataset is a window.
      num_windows = int(num_features / window_size)
      for window_idx in range(num_windows):
        windowed_data = [None] * (window_size * dims)
        for data_idx in range(window_size):
          for data_dim_idx in range(dims):
            windowed_data[(data_idx * dims) +
                          data_dim_idx] = feature_list.feature[
                              (window_idx * window_size) +
                              data_idx].float_list.value[data_dim_idx]
        labeled_dataset[label_id].append(windowed_data)

for label_id, items in labeled_dataset.items():
  labeled_dataset[label_id] = np.array(items)

In [ ]:
# Split data between training, validation, and testing
label_ids = labeled_dataset.keys()
dataset_split_names = {'train', 'validation', 'test'}
np_split_dataset = {'train': {}, 'validation': {}, 'test': {}}
for label_id in label_ids:
  np_split_dataset['test'][label_id], temp = train_test_split(
      labeled_dataset[label_id], test_size=PERCENT_TEST)
  np_split_dataset['train'][label_id], np_split_dataset['validation'][
      label_id] = train_test_split(
          temp, test_size=PERCENT_TRAIN / (PERCENT_TRAIN + PERCENT_VALIDATION))

# Add labels and shuffle the data.
tf_split_datasets = {}
for split_name in dataset_split_names:
  examples = []
  labels = []
  for label_id in label_ids:
    examples.extend(np_split_dataset[split_name][label_id])
    labels.extend(
        np.full(len(np_split_dataset[split_name][label_id]), label_id))
  tf_split_datasets[split_name] = tf.data.Dataset.from_tensor_slices(
      (np.array(examples),
       np.array(labels))).shuffle(len(examples)).batch(DATASET_BATCH_SIZE)

In [ ]:
# Binary training
assert len(label_ids) == 2

# Create model
binary_model = keras.Sequential([
    keras.layers.Flatten(input_shape=labeled_dataset[0][0].shape),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
binary_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.TruePositives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalseNegatives()
    ])

# Train model.
binary_model.fit(
    x=tf_split_datasets['train'],
    epochs=15,
    validation_data=tf_split_datasets['validation'])

In [ ]:
# Test model
test_loss, test_acc, true_pos, false_pos, true_neg, false_neg = binary_model.evaluate(
    tf_split_datasets['test'], verbose=1)

In [ ]:
# Save model
binary_model.save(SAVED_MODEL_FOLDER)

In [ ]:
# Multiclass training

# Create model
multiclass_model = keras.Sequential([
    keras.layers.Flatten(input_shape=labeled_dataset[0][0].shape),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(len(label_ids), activation='softmax'),
])
multiclass_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'])

# Train model.
multiclass_model.fit(
    x=tf_split_datasets['train'],
    epochs=15,
    validation_data=tf_split_datasets['validation'])

In [ ]:
# Test model
test_loss, test_acc = multiclass_model.evaluate(tf_split_datasets['test'], verbose=1)

In [ ]:
# Save model
multiclass_model.save(SAVED_MODEL_FOLDER)